In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 로드

In [82]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import numpy as np
import pandas as pd

In [84]:
train_original = pd.read_excel('/content/drive/MyDrive/데이터 분석/07. 1차 프로젝트/02. Train_Test Data/train.xlsx')
print(train_original.shape)
train_original.head()

(50000, 2)


,label,text
0,1,Almost perfect.My first bidet with bells and w...
1,1,"Soothing: Bought this for my mom, she is Japan..."
2,0,blond phone: nokia 8801 is a very attractive p...
3,0,Saw No Change - Zero - Absolutely None!: I am ...
4,0,"catchy, true... But nothing more: There is one..."


In [85]:
test_original = pd.read_excel('/content/drive/MyDrive/데이터 분석/07. 1차 프로젝트/02. Train_Test Data/bidet_test_data.xlsx')
print(test_original.shape)
test_original.head()

(13090, 2)


,label,text
0,1,Gives Toto a run for your money..Excellent sol...
1,1,Amazing.After owning the Bio Bidet. I can’t se...
2,1,Warning - looong review.Installation
3,1,Life changing!.We originally purchased this un...
4,1,A wonderful addition to our house! .I have ...


## 카피본

In [86]:
train = train_original.copy()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   50000 non-null  int64 
 1   text    50000 non-null  object
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [87]:
test = test_original.copy()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13090 entries, 0 to 13089
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   13090 non-null  int64 
 1   text    13090 non-null  object
dtypes: int64(1), object(1)
memory usage: 204.7+ KB


In [88]:
# 데이터 로딩
train_data = train['text']  # 훈련 데이터셋
train_labels = train['label']  # 훈련 데이터셋 라벨
test_data = test['text']  # 테스트 데이터셋
test_labels = test['label']  # 테스트 데이터셋 라벨

# 전처리

## 소문자화, 불용어/구두점 제거

In [89]:
train_data = [str(text) for text in train_data]
test_data = [str(text) for text in test_data]

import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # 불용어 사전 다운로드
nltk.download('wordnet')
stop_words = stopwords.words('english')


def preprocess_text(text):  
    # text = text.lower() # 소문자 변환
    # text = re.sub(r'\.+', ' ', text)  # 중복 온점 제거
    
    # punctuations = '\"#$%*+,/<=>@[\]^_`\\\\{|}?!~:;().'
    # for p in punctuations :
    #     text = text.replace(p, ' ')
    
    # text = text.replace('&', 'and')
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = re.sub(r'\s+', ' ', text)  # 중복 공백 제거
#     text = re.sub(r'(\w)(\1{1,})', r'\1', text) # 알파벳 3회이상 반복시 1개로
      
#     # 구두점 제거
#     text = ''.join(char for char in text if char not in punctuations)
    # 불용어 제거
#     text = ' '.join(word for word in text.split() if word not in stop_words)


#     return ' '.join(words)
    return text.strip()


# train_data와 test_data에 대해 전처리 수행
preprocessed_train_data = [preprocess_text(text) for text in train_data]
preprocessed_test_data = [preprocess_text(text) for text in test_data]
total_texts = preprocessed_test_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 토큰화 및 어휘집 생성

- 텐서(Tensor) : 다차원 배열을 표현하는 자료형 : 벡터, 행렬, 그리고 이들의 고차원 일반화된 것
- 텐서화 : 입력 데이터를 모델에 맞는 텐서 자료형으로 변환하는 과정 : 대부분 딥 러닝 모델에 입력 데이터로 사용하기 위해 수행
- 텍스트 데이터의 텐서화 : 
  - 1. 텍스트 데이터를 토큰화하여 각 단어를 고유한 숫자로 맵핑 : 맵핑은 일반적으로 어휘집(vocabulary)이라고 불리는 고유한 숫자와 단어의 쌍으로 이루어진 사전을 이용
  - 2. 각 단어를 고유한 숫자로 맵핑한 후, 이를 텐서 자료형으로 변환 : 주로 사용되는 텐서 자료형으로는 파이토치(PyTorch)의 torch.Tensor나 텐서플로(TensorFlow)의 tf.Tensor 등이 있음
- 예시 :
  - 파이토치 : torchtext.data.Field를 사용하여 토큰화를 수행하고, torch.utils.data.DataLoader를 사용하여 텐서로 변환
  - 텐서플로 : tf.keras.preprocessing.text.Tokenizer를 사용하여 토큰화를 수행하고, tf.data.Dataset을 사용하여 텐서로 변환

In [90]:
import torch # 파이토치 이용
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 토큰화 및 어휘집 생성
def tokenize_data(reviews, max_length=128):

    # BERT 모델의 사전학습시 학습한 tokenizer 불러오기
    # BERT : Bidirectional Encoder Representations from Transformers
    # BERT : 구글에서 개발한 언어 모델로 기존 언어 모델과 달리 양방향으로 훈련하므로 문맥을 참고하여 문장 의미 이해 가능
    # BertTokenizer : Hugging Face의 Transformers 라이브러리에서 제공하는 BERT(Bidirectional Encoder Representations from Transformers) 모델의 토큰화(tokenization) 기능을 담당하는 클래스
    # from_pretrained() : 미리 학습된 BERT 모델을 로드 및 해당 모델의 토크나이저를 가져오는 메소드
    # bert-base-uncased : 전체 단어를 소문자로 처리하는 uncased (대소문자를 구문하지 않는) 모델
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    input_ids, attention_masks = [], []

    for review in reviews:
        # encode_plus 메서드를 이용해 리뷰를 토큰화하고, 인코딩하여 입력 문장을 BERT 모델이 이해할 수 있는 형태의 텐서 데이터로 변환해주는 역할을 함
        encoded_dict = tokenizer.encode_plus(
            # encode_plus : 입력 데이터를 인코딩하는 함수 중 하나로서, 문자열로 된 문장을 입력으로 받아서 BERT 모델의 입력 형식에 맞게 인코딩된 데이터를 반환함
            review, # 인코딩할 문자열 = 리뷰
            add_special_tokens=True,  # CLS, SEP 토큰 추가 여부
            max_length=max_length, # 문장의 최대 길이최대 길이
            padding='max_length', # 길이가 부족한 경우 최대 길이로 padding
            return_attention_mask=True, # 어텐션 마스크 리턴 여부
            return_tensors="pt", # 반환할 텐서의 타입 : 파이토치 텐서로 리턴
            truncation=True # 길이가 너무 길면 문장을 잘라서 처리
        )
        
        # encode_plus는 문장을 "토큰 ID, 어텐션 마스크, 세그먼트 ID" 등의 입력 데이터 형태로 변환
        # -> encode_plus 함수의 반환 값은 input_ids, attention_mask, token_type_ids 등으로 구성
        # input_ids : 인코딩된 토큰들을 식별하기 위한 고유한 ID 값
        # attention_mask : 각 토큰이 패딩이 아니라 실제 입력 값인지 여부를 나타내는 마스크
        # token_type_ids : BERT 모델이 두 개의 문장을 처리할 때 첫 번째 문장과 두 번째 문장의 경계를 나타내는 ID 값

        # print(encoded_dict["input_ids"].size())
        # print(encoded_dict["attention_mask"].size())

        # 인코딩된 토큰과 어텐션 마스크를 각각 리스트에 저장
        input_ids.append(encoded_dict["input_ids"])
        # input_ids : 각 리뷰를 토큰화한 결과를 숫자로 인코딩한 텐서
        attention_masks.append(encoded_dict["attention_mask"])
        # attention_masks : 어텐션 마스크 텐서서

    # 리스트의 모든 텐서를 결합(concatenate)
    # torch.cat : PyTorch에서 tensor들을 이어붙이는 역할 : 더욱 효율적으로 모델 학습 수행 가능
    # 첫 번째 인자 : tensor들이 저장된 리스트
    # dim : 파라미터로 이어붙이는 방향을 설정 (0:세로 방향으로 쌓아올림)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

## 데이터셋 및 데이터 로더 생성

In [91]:
from torch.utils.data import DataLoader, TensorDataset
# TensorDataset : 입력과 출력 데이터를 함께 묶어 하나의 데이터셋으로 만드는 PyTorch 제공 기능
# DataLoader : 데이터셋을 배치 크기(batch size)에 맞게 분할하여 각 배치(batch)를 반환 : 배치 단위로 데이터를 처리하면 전체 데이터를 한꺼번에 처리할 때 보다 메모리 사용량 절약 가능

# 데이터셋 및 데이터 로더 생성
def create_dataset(input_ids, attention_masks, labels, batch_size=64):
    dataset = TensorDataset(input_ids, attention_masks, labels)
    # 입력 데이터와 라벨을 인자로 받아, 두 개의 데이터셋을 결합해 하나의 데이터셋으로 결합
    # 위 함수를 통해 만들어진 데이터셋은 DataLoader 클래스에 전달되어 학습, 검증, 테스트 데이터셋으로 활용
    # input_ids: 토큰화된 입력 데이터의 인덱스를 담은 PyTorch 텐서
    # attention_masks: 패딩된 데이터의 영역을 표시하는 토큰 마스크 텐서
    # labels: 데이터에 대한 라벨을 포함하는 PyTorch 텐서
    data_loader = DataLoader(dataset, batch_size=batch_size)
    # batch_size: 데이터를 나눌 배치 크기

    return data_loader

# 모델

## 모델 생성

In [92]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
# BertForSequenceClassification : BERT 모델을 기반으로 한 문장 분류 모델
# 사전 훈련된 BERT 모델을 사용하여 텍스트 입력 시퀀스의 특성을 추출하고, 이를 이용하여 해당 시퀀스가 특정 클래스에 속하는지 예측하는 분류 모델을 구현
# 이 모델은 사전 훈련된 BERT 모델을 불러와서 사용하기 때문에, 이전에 학습된 BERT 모델의 가중치를 가져와야 하므로, 모델 호출시 from_pretrained() 함수를 사용하는 것것

# 모델 불러오기 및 설정
def load_model():
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    # Hugging Face에서 제공하는 bert-base-uncased 모델을 불러오고
    # 분류 문제의 라벨 개수를 num_labels 파라미터로 전달
    # for param in model.parameters():
    #     param.requires_grad = True # 모든 레이어를 미세 조정   
    
    # GPU 사용 가능 여부 확인 후 모델을 GPU로 이동
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    return model

## 모델 학습

In [93]:
import torch # 파이토치 이용
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
# AdamW : Adam 옵티마이저의 변형으로, 가중치 감쇠(weight decay)를 추가하여 L2 규제를 사용하는 것과 비슷한 효과를 낼 수 있음
# optimizer란? : 손실함수의 값을 줄여나가는 학습 방법이자 경사하강법 알고리즘
# Adam Optimizer란 ? : RMSprop과 Momentum을 합친듯한 방법으로 방향과 학습률 모두를 잡기위한 방법
# get_linear_schedule_with_warmup : 학습률을 처음 몇 번의 스텝에서 작은 값으로 시작하여 서서히 증가시키는 방식의 스케줄러
# train 모드와 eval 모드 : 모델 학습 및 평가 시 모델의 동작 방식을 나타내는 모드
# train 모드 : 학습을 위한 준비 작업 수행 : 모델은 gradient를 계산하고 역전파(backpropagation)를 수행하여 가중치를 업데이트하며 모델에서 사용하는 일부 레이어의 동작 방식이 변경될 수 있음 ex) 드롭아웃(dropout) 레이어는 train 모드에서는 일부 노드를 랜덤하게 제거하여 과적합을 방지하는 역할
# eval 모드 : 모델의 가중치를 고정시켜 평가 수행 : 일부 레이어의 동작 방식이 train 모드와 다를 수 있음 ex) 드롭아웃 레이어는 eval 모드에서는 모든 노드를 유지하여 출력을 계산
# Dropout과 BatchNormalization : 신경망 모델에서 Overfitting을 방지하기 위한 regularization 방법
# Dropout : 모델을 일반화 시킴 :신경망 모델의 일부 뉴런을 무작위로 선택하여 출력을 0으로 만듬 -> 이를 통해 학습 중에 뉴런들이 서로 의존하는 경향을 줄이고, 더욱 강건한 모델을 만들어 Overfitting을 방지
# BatchNormalization : 입력 데이터를 정규화하여 학습의 안정성 증가 : 입력 데이터의 평균과 분산을 정규화하여, 신경망 모델에서 발생하는 Internal Covariate Shift 문제를 해결 -> 이를 통해 모델의 학습 속도를 높이고, 일반화 성능을 향상시키며 Gradient vanishing 문제를 다소 완화
# Dropout과 BatchNormalization은 학습과정에서만 적용하고, 검증과 추론과정에서는 미적용
# 따라서 모델 검증 및 추론 단계 이전에 모델을 eval mode로 변경하여 일반화 능력을 평가해야함

# 모델 학습
def train(model, train_loader, val_loader, epochs=4, learning_rate=2e-5, patience=5):
    # train_loader로부터 미니배치를 받아 모델 학습
    # val_loader로부터 미니배치를 받아 모델 검증
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    # AdamW optimizer 사용, learning_rate는 2e-5로 설정
    # AdamW Optimizer의 구현이 deprecation될 예정이라는 warning massage에 따라 PyTorch에서 제공하는 torch.optim.AdamW를 사용한 것
    total_steps = len(train_loader) * epochs
    # 전체 학습 횟수 : train_loader 길이 * epochs로 설정
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    # scheduler : 학습률을 동적으로 조절
    # warmup_steps를 0으로 설정하고 total_steps만큼 스텝마다 학습률 lr을 감소시키도록 설정
    # -> 처음 몇 스텝 동안은 학습률을 점진적으로 증가시켜 안정적인 학습 시작 가능

    best_val_accuracy = 0  # 2. best_val_accuracy 초기화
    no_improvement_counter = 0  # 2. no_improvement_counter 초기화

    for epoch in range(epochs):
        
        # 1. 각 epoch마다 모델을 train mode로 설정
        model.train() # train 모드로 설정시 train 데이터를 이용하여 학습
        total_loss = 0

        for batch in train_loader:
            # train_loader로부터 미니배치를 받아 모델을 학습
            input_ids, attention_masks, labels = tuple(t.to(device) for t in batch)
            # input_ids, attention_masks, labels를 device로 보내서 계산
            model.zero_grad() # gradient 초기화 (gradient:손실(loss)의 기울기)
            outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
            # outputs : model을 이용하여 input_ids, attention_masks, labels를 입력으로 하여 outputs를 계산
            # token_type_ids : BERT 모델이 두 개의 문장을 처리할 때 첫 번째 문장과 두 번째 문장의 경계를 나타내는 ID 값

            # forward 연산 수행
            loss = outputs[0] # 각 loss 계산
            total_loss += loss.item() # 총 loss 계산

            # backward 연산 수행
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # gradient clipping을 통한 gradient exploding 방지
            optimizer.step() # optimizer의 step 수행
            scheduler.step() # scheduler의 step 수행

        avg_train_loss = total_loss / len(train_loader)
        # 각 epoch마다의 avg_train_loss 계산



        # 2. 각 epoch마다 모델을 train mode로 설정
        # 모델의 모드를 train -> eval로 변경하여 검증 데이터에 대한 accuracy 계산
        # eval 모드에서는 학습을 하지 않으므로 dropout을 비롯한 regularization 기법이 적용되지 않음
        # Dropout이나 BatchNormalization 등 학습과 검증 단계에서 다르게 적용되는 요소를 설정
        model.eval()
        total_eval_accuracy = 0

        # validation 데이터를 이용하여 검증
        for batch in val_loader:
            input_ids, attention_masks, labels = tuple(t.to(device) for t in batch)
            # input_ids, attention_masks, labels를 device로 보내서 계산
            with torch.no_grad(): # gradient 계산 안함
                # forward 연산 수행
                outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
            
            # logits 계산
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
             # label_ids 계산
            label_ids = labels.to("cpu").numpy()
            # accuracy 계산
            total_eval_accuracy += accuracy_score(label_ids, logits.argmax(axis=-1))

        avg_val_accuracy = total_eval_accuracy / len(val_loader)
        # 각 epoch마다의 검증 정확도 avg_val_accuracy 계산
        # 이 값이 높을수록 모델이 더 잘 동작하는 것을 의미

        # 각 epoch의 training loss와 validation accuracy 출력
        print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}")



        # Early Stopping을 위한 조건 검사
        if avg_val_accuracy > best_val_accuracy:
            best_val_accuracy = avg_val_accuracy
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1

        # Early Stopping 조건 충족 시 학습 종료
        if no_improvement_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs due to no improvement in validation accuracy.")
            break

## 모델 평가

In [94]:
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

# 모델 평가 (test dataset에 대한 성능 평가)
def evaluate(model, test_loader):
    model.eval()
    # 모델의 모드를 evaluation mode로 변경
    total_eval_accuracy = 0
    total_preds = []
    total_labels = []
    total_probs = []  # 추가

    for batch in test_loader:
        # test dataset의 미니배치를 하나씩 가져옴
        input_ids, attention_masks, labels = tuple(t.to(device) for t in batch)
        # 미니배치에 포함된 input_ids, attention_masks, labels를 device(GPU 또는 CPU)로 전송
        with torch.no_grad():
            # gradient 계산을 수행하지 않음으로서 메모리 사용을 최적화 
            outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
            # 모델의 출력값 계산

        logits = outputs[0]
        # logits : 모델이 예측한 클래스의 점수
        probs = F.softmax(logits, dim=-1)
        # softmax 함수를 사용하여 확률로 변환
        logits = logits.detach().cpu().numpy()
        probs = probs.detach().cpu().numpy()
        # 모델의 출력값에서 gradient를 분리하고, CPU로 데이터를 전송한 후 numpy 배열로 변환
        label_ids = labels.to("cpu").numpy()
        # label도 numpy 배열로 변환
        total_preds.append(logits.argmax(axis=-1))
        total_probs.append(probs[:, 1])  # 추가
        total_labels.append(label_ids)


        total_eval_accuracy += accuracy_score(label_ids, logits.argmax(axis=-1))
        # accuracy_score : 각 미니배치에서 모델이 예측한 레이블과 실제 레이블(label_ids)을 비교하여 정확도를 계산

    avg_test_accuracy = total_eval_accuracy / len(test_loader)
    # 전체 테스트 데이터셋에 대한 평균 정확도를 계산
    print(f"Test Accuracy: {avg_test_accuracy:.4f}")
    # 평균 정확도 출력
    

    # 실제 레이블, 모델이 예측한 레이블, 확률, 텍스트를 모아서 표로 출력
    preds = np.concatenate(total_preds, axis=0)
    probs = np.concatenate(total_probs, axis=0)
    labels = np.concatenate(total_labels, axis=0)
    texts = total_texts
    eval_result_df = pd.DataFrame({'Text': texts, 'Label': labels, 'Prediction': preds, 'Probability': probs})
    print(eval_result_df.head(10))


    return eval_result_df

# 평가

## 적용

In [95]:
from sklearn.model_selection import train_test_split

# train 데이터와 validation 데이터를 분리
train_data, val_data, train_labels, val_labels = train_test_split(preprocessed_train_data, train_labels, test_size=0.2, random_state=42)

# 전처리된 데이터 토큰화
train_input_ids, train_attention_masks = tokenize_data(train_data, max_length=128)
val_input_ids, val_attention_masks = tokenize_data(val_data, max_length=128)
test_input_ids, test_attention_masks = tokenize_data(preprocessed_test_data)

# 라벨을 텐서로 변환
train_labels_tensor = torch.tensor(train_labels.values)
val_labels_tensor = torch.tensor(val_labels.values)
test_labels_tensor = torch.tensor(test_labels.values)

# 데이터 로더 생성
train_loader = create_dataset(train_input_ids, train_attention_masks, train_labels_tensor, batch_size=32)
val_loader = create_dataset(val_input_ids, val_attention_masks, val_labels_tensor, batch_size=32)
test_loader = create_dataset(test_input_ids, test_attention_masks, test_labels_tensor)

# 모델 불러오기
device = torch.device("cuda")
model = load_model()

# 모델 학습
train = train(model, train_loader, val_loader, epochs=4, learning_rate=2e-5)

# 모델 평가
eval_result_df = evaluate(model, test_loader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 1, Training Loss: 0.1671, Validation Accuracy: 0.9493
Epoch: 2, Training Loss: 0.0854, Validation Accuracy: 0.9516
Epoch: 3, Training Loss: 0.0469, Validation Accuracy: 0.9513
Epoch: 4, Training Loss: 0.0285, Validation Accuracy: 0.9521
Test Accuracy: 0.9839
                                                Text  Label  Prediction  \
0  Gives Toto a run for your money..Excellent sol...      1           1   
1  Amazing.After owning the Bio Bidet. I can’t se...      1           1   
2               Warning - looong review.Installation      1           1   
3  Life changing!.We originally purchased this un...      1           1   
4  A wonderful addition to our house! .I have alw...      1           1   
5  Great coverage and the warranty is just great!...      1           1   
6  Great product..Great product. Easy to install....      1           1   
7  Facile à installer, répond à tous nos besoins,...      1           1   
8  So fresh and so clean, clean❤️.This is great. ...      1

In [106]:
eval_result_df1 = eval_result_df.copy()
eval_result_df1

,Text,Label,Prediction,Probability
0,Gives Toto a run for your money..Excellent sol...,1,1,0.999522
1,Amazing.After owning the Bio Bidet. I can’t se...,1,1,0.998895
2,Warning - looong review.Installation,1,1,0.990475
3,Life changing!.We originally purchased this un...,1,1,0.999522
4,A wonderful addition to our house! .I have alw...,1,1,0.999476
...,...,...,...,...
13085,"Great Value, Awesome Features.I bought this fo...",1,1,0.999580
13086,Best gift ever ! .A friend bought me a biobide...,1,1,0.999397
13087,"Best Buy last months!.Wow, how did we even kno...",1,1,0.999456
13088,Excellent product.I am extremely happy with th...,1,1,0.999532


In [109]:
eval_result_df1 = pd.DataFrame(eval_result_df1)
eval_result_df1

,Text,Label,Prediction,Probability
0,Gives Toto a run for your money..Excellent sol...,1,1,0.999522
1,Amazing.After owning the Bio Bidet. I can’t se...,1,1,0.998895
2,Warning - looong review.Installation,1,1,0.990475
3,Life changing!.We originally purchased this un...,1,1,0.999522
4,A wonderful addition to our house! .I have alw...,1,1,0.999476
...,...,...,...,...
13085,"Great Value, Awesome Features.I bought this fo...",1,1,0.999580
13086,Best gift ever ! .A friend bought me a biobide...,1,1,0.999397
13087,"Best Buy last months!.Wow, how did we even kno...",1,1,0.999456
13088,Excellent product.I am extremely happy with th...,1,1,0.999532


In [116]:
eval_result_df1.to_excel('/content/drive/MyDrive/데이터 분석/07. 1차 프로젝트/03. 머신러닝_딥러닝/sentiment.xlsx')

## 모델 저장 

In [97]:
model.save_pretrained('./model')

## 이미지 저장

In [98]:
from IPython.display import Image
Image(filename=filename)

NameError: ignored